<a href="https://colab.research.google.com/github/thisisanu/online_deep_learning/blob/main/homework3/Homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
torch.cuda.is_available()
!nvidia-smi
!pip install numpy torch torchvision tqdm matplotlib

Tue Nov 11 00:03:06 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P8             13W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [35]:
import os
import shutil

# Ensure we are in /content
os.chdir("/content")
print("Current directory:", os.getcwd())

# If a folder named homework2 exists, remove it completely
if os.path.exists("online_deep_learning"):
    shutil.rmtree("online_deep_learning")
    print("Old online_deep_learning folder removed.")
else:
    print("No existing online_deep_learning folder found.")

# Remove the .git folder to disconnect from the current repository
!rm -rf .git
!git clone https://github.com/thisisanu/online_deep_learning.git
%cd online_deep_learning/homework3


Current directory: /content
Old online_deep_learning folder removed.
Cloning into 'online_deep_learning'...
remote: Enumerating objects: 821, done.
remote: Counting objects: 100% (41/41), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 821 (delta 27), reused 0 (delta 0), pack-reused 780 (from 1)
Receiving objects: 100% (821/821), 3.35 MiB | 23.35 MiB/s, done.
Resolving deltas: 100% (574/574), done.
/content/online_deep_learning/homework3


In [ ]:

#download the datasets by running the following command from the main directory
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/classification_data.zip -o ./classification_data.zip && unzip -qo classification_data.zip
!curl -s -L https://www.cs.utexas.edu/~bzhou/dl_class/drive_data.zip -o ./drive_data.zip && unzip -qo drive_data.zip

In [4]:
import os
# Paths
train_path = './classification_data/train'
val_path = './classification_data/val'

def print_data_overview(directory, name):
    print(f"\n{name} ({directory}):")
    if os.path.exists(directory):
        items = os.listdir(directory)
        print(f"Total items: {len(items)}")
    else:
        print("Directory does not exist!")

# Print directories
print_data_overview(train_path, "Training data")
# List train folders
train_folders = os.listdir(train_path)
print("Train folders:")
print(train_folders[:10])  # show first 10
!ls classification_data/train/*.csv
!head classification_data/train/labels.csv

print_data_overview(val_path, "Validation data")
# List val folders
val_folders = os.listdir(val_path)
print("Validation folders:")
print(val_folders[:10])  # show first 10
!ls classification_data/val/*.csv
!head classification_data/val/labels.csv


Training data (./classification_data/train):
Total items: 21001
Train folders:
['05035.jpg', '04562.jpg', '18590.jpg', '00733.jpg', '20676.jpg', '10804.jpg', '07942.jpg', '14084.jpg', '04333.jpg', '11031.jpg']
classification_data/train/labels.csv
file,label,track
00001.jpg,background,abyss
00002.jpg,background,abyss
00003.jpg,background,abyss
00004.jpg,background,abyss
00005.jpg,background,abyss
00006.jpg,background,abyss
00007.jpg,background,abyss
00008.jpg,background,abyss
00009.jpg,background,abyss

Validation data (./classification_data/val):
Total items: 9001
Validation folders:
['05035.jpg', '04562.jpg', '00733.jpg', '07942.jpg', '04333.jpg', '07378.jpg', '02894.jpg', '04960.jpg', '06734.jpg', '08624.jpg']
classification_data/val/labels.csv
file,label,track
00001.jpg,background,candela_city
00002.jpg,background,candela_city
00003.jpg,background,candela_city
00004.jpg,background,candela_city
00005.jpg,background,candela_city
00006.jpg,background,candela_city
00007.jpg,background,

**Reasoning**:
The previous steps successfully created the `train_classification.py` file. Now, to verify its functionality and ensure the initial setup components (device config, data transforms, data loading) are correctly integrated, I will execute the script. This will confirm that the file is executable and performs the expected setup operations.



In [5]:
import sys
import os

print("Verifying train_classification.py setup...")
!python -m homework.train_classification
print("Verification complete.")

Verifying train_classification.py setup...
Using device: cuda
[train_classification] Looking for labels.csv at: ./classification_data/train/labels.csv
[get_class_names] Looking for labels.csv at: ./classification_data/train/labels.csv
Found classes: ['background', 'bomb', 'kart', 'nitro', 'pickup', 'projectile']

Epoch 1/10
--------------------
train Loss: 0.9362 Acc: 0.6783
val Loss: 0.4300 Acc: 0.8507

Epoch 2/10
--------------------
train Loss: 0.3270 Acc: 0.8908
val Loss: 0.4269 Acc: 0.8430

Epoch 3/10
--------------------
train Loss: 0.2265 Acc: 0.9259
val Loss: 0.2899 Acc: 0.8920

Epoch 4/10
--------------------
train Loss: 0.1826 Acc: 0.9399
val Loss: 0.2565 Acc: 0.9022

Epoch 5/10
--------------------
train Loss: 0.1584 Acc: 0.9485
val Loss: 0.3399 Acc: 0.8857

Epoch 6/10
--------------------
train Loss: 0.1452 Acc: 0.9522
val Loss: 0.2617 Acc: 0.9113

Epoch 7/10
--------------------
train Loss: 0.1334 Acc: 0.9567
val Loss: 0.3643 Acc: 0.8817

Epoch 8/10
--------------------
tr

In [37]:
import sys
import os

print("Verifying train_detection.py setup...")
!python -m homework.train_detection
print("Verification complete.")

Verifying train_detection.py setup...
Loaded 8000 samples from 16 episodes
Loaded 2000 samples from 4 episodes
Epoch 1/30, Seg Loss: 0.2298, Depth Loss: 0.1085
Val IoU: 0.3240, Val Depth MAE: 0.0618, Val Depth MAE (boundary): 0.0870
Saved best model with val IoU: 0.3240 -> /content/online_deep_learning/homework3/homework/detector.th
Epoch 2/30, Seg Loss: 0.1428, Depth Loss: 0.0600
Val IoU: 0.3240, Val Depth MAE: 0.0593, Val Depth MAE (boundary): 0.1094
Epoch 3/30, Seg Loss: 0.1374, Depth Loss: 0.0557
Val IoU: 0.3240, Val Depth MAE: 0.0519, Val Depth MAE (boundary): 0.0687
Epoch 4/30, Seg Loss: 0.1107, Depth Loss: 0.0511
Val IoU: 0.3240, Val Depth MAE: 0.0472, Val Depth MAE (boundary): 0.0482
Epoch 5/30, Seg Loss: 0.1007, Depth Loss: 0.0481
Val IoU: 0.3241, Val Depth MAE: 0.0446, Val Depth MAE (boundary): 0.0432
Saved best model with val IoU: 0.3241 -> /content/online_deep_learning/homework3/homework/detector.th
Epoch 6/30, Seg Loss: 0.0964, Depth Loss: 0.0460
Val IoU: 0.3243, Val Depth

In [37]:
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git add *.py
!git commit -m "update"
!git push origin main

ada3488.zip              drive_data/     Homework3.ipynb
bundle.py                drive_data.zip  README.md
classification_data/     grader/         requirements.txt
classification_data.zip  homework/
On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	ada3488.zip
	classification_data.zip
	classification_data/
	drive_data.zip
	drive_data/
	grader/__pycache__/
	grader/datasets/__pycache__/
	homework/__pycache__/
	homework/classifier.th
	homework/datasets/__pycache__/

nothing added to commit but untracked files present (use "git add" to track)
Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@32bd00ffe7c6.(none)')
fatal: could not read U

In [18]:
import os
zip_file = 'ada3488.zip'
if os.path.exists(zip_file):
    os.remove(zip_file)
    print(f"Removed existing {zip_file}")
else:
    print(f"{zip_file} not found, no need to remove.")

ada3488.zip not found, no need to remove.


In [38]:
!pwd
!python3 bundle.py homework ada3488
!python3 -m grader ada3488.zip -vv --disable_color


/content/online_deep_learning/homework3
train_classification.py
models.py
metrics.py
checkpoints
detector.th
__init__.py
datasets
train_detection.py
checkpoints/detector_epoch28.pt
checkpoints/detector_epoch3.pt
checkpoints/detector_epoch21.pt
checkpoints/detector_epoch22.pt
checkpoints/detector_epoch17.pt
checkpoints/detector_epoch18.pt
checkpoints/detector_epoch7.pt
checkpoints/detector_epoch6.pt
checkpoints/detector_epoch2.pt
checkpoints/detector_epoch25.pt
checkpoints/detector_epoch10.pt
checkpoints/detector_epoch12.pt
checkpoints/detector_epoch8.pt
checkpoints/detector_epoch24.pt
checkpoints/detector_epoch30.pt
checkpoints/detector_epoch20.pt
checkpoints/detector_epoch13.pt
checkpoints/detector_epoch27.pt
checkpoints/detector_epoch23.pt
checkpoints/detector_epoch15.pt
checkpoints/detector_epoch4.pt
checkpoints/detector_epoch9.pt
checkpoints/detector_epoch29.pt
checkpoints/detector_epoch16.pt
checkpoints/detector_epoch1.pt
checkpoints/detector_epoch19.pt
checkpoints/detector_epoch5